In [48]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re

# 다나와 사이트 크롤링

### 크롬 드라이버 경로 지정

In [43]:
path = 'C:/Users/mbnb8/Downloads/chromedriver.exe'

In [44]:
driver = webdriver.Chrome(path) 
#여기서 우리가 제어하고자 하는 chrom 창이 뜸.
#그거 끄면 안됨

### 크롤링 하고자 하는 링크 get

In [45]:
driver.get('http://cws.danawa.com/cmRecommend/cm_List.php?nSite=5&logger_kw=cmnotebook_shoppinghelp')
time.sleep(1)
#실행하면 해당 링크가 크롬창으로 열린다

### 1페이지 부터 10페이지 까지 끌어오기

In [89]:
text = []
user = []
for i in range(1,10+1): 
    #1페이지에서 2페이지갈 때는 i:1, 2에서 3갈때는 i:2, 9에서 10갈때는 i:9
    text_page = driver.find_elements_by_class_name("comment")
    user_page = driver.find_elements_by_class_name("user")
    num_text_in_page = len(text_page)
    
    for j in range(num_text_in_page):
        text += [text_page[j].text]
        user += [user_page[j].text]
    
    driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/div/div/a["""+str(i)+"""]""")[0].click()
    time.sleep(2)

for h in range(30): #30으로 핳면 310페이지 까지
    for i in range(2,11+1):
        text_page = driver.find_elements_by_class_name("comment")
        user_page = driver.find_elements_by_class_name("user")
        num_text_in_page = len(text_page)
    
        for j in range(num_text_in_page):
            text += [text_page[j].text]
            user += [user_page[j].text]
    
        driver.find_elements_by_xpath("""//*[@id="qna_List"]/ul/div/div/a["""+str(i)+"""]""")[0].click()
        time.sleep(2)


In [92]:
data = pd.DataFrame({'user': user,'comment': text})

In [94]:
#data.to_csv('data.csv',encoding='ms949')

# 크롤링한 데이터 텍스트 처리

In [98]:
#data = pd.read_csv('data.csv',encoding='ms949')

In [99]:
data.head()

,Unnamed: 0,user,comment
0,0,L1 쥬쥬쥬쥬 (jjooo77) 님 | 203.229.xxx.71 | 2019.04.27,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n-문서편집,넷플릭스시청,인터..."
1,1,L1 새얀이 (nasujin95) 님 | 223.33.xxx.112 | 2019.0...,1.동영상 편집 / 프리미어 프로\r\n2. 100~150 만원\r\n3. 숫자 키...
2,2,L1 오늘어때 (K0081652057) 님 | 223.62.xxx.132 | 201...,1. 사용 용도 업무용 엑셀 워드 캐드\r\n2. 원하는 가격대 150만 이하\r\...
3,3,L1 tn4mwqz8phb (aredu) 님 | 61.101.xxx.155 | 20...,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n인터넷(유튜브), 한글, M..."
4,4,L1 sootaejy (sootaejy) 님 | 147.47.xxx.230 | 20...,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n오피스, 간단한 프로그래밍 ..."


### user 열에서 0000.00.00의 날짜 정보만 뽑아오기

In [100]:
data['date'] = data['user'].str.extract('(\d{4}\.\d{2}\.\d{2})')

In [102]:
data.head()

,Unnamed: 0,user,comment,date
0,0,L1 쥬쥬쥬쥬 (jjooo77) 님 | 203.229.xxx.71 | 2019.04.27,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n-문서편집,넷플릭스시청,인터...",2019.04.27
1,1,L1 새얀이 (nasujin95) 님 | 223.33.xxx.112 | 2019.0...,1.동영상 편집 / 프리미어 프로\r\n2. 100~150 만원\r\n3. 숫자 키...,2019.04.27
2,2,L1 오늘어때 (K0081652057) 님 | 223.62.xxx.132 | 201...,1. 사용 용도 업무용 엑셀 워드 캐드\r\n2. 원하는 가격대 150만 이하\r\...,2019.04.27
3,3,L1 tn4mwqz8phb (aredu) 님 | 61.101.xxx.155 | 20...,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n인터넷(유튜브), 한글, M...",2019.04.26
4,4,L1 sootaejy (sootaejy) 님 | 147.47.xxx.230 | 20...,"1. 사용 용도 (사용하는 프로그램/게임명 기재)\r\n오피스, 간단한 프로그래밍 ...",2019.04.26


### comment 열에서 '1. 사용 용도'에 해당하는 텍스트만 가져오기

In [103]:
len = data.shape[0]
purpose = []
for i in range(len):
    a = data['comment'][i].partition('2.')[0] #'2.' 이전의 문자열 가져오기
    b = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',a) # 특수문자 제거
    c = re.sub('\r\n',' ',b) # \r\n 제거
    purpose += [c]

### word 단위로 쪼개기

In [104]:
purpose_detail = []
for pur in purpose:
    purpose_detail += [pur.split()]

purpose_df = pd.DataFrame({'purpose': purpose_detail})

data1 = pd.concat([data,purpose_df],axis=1)[['date','purpose']]

In [105]:
data1.head()

,date,purpose
0,2019.04.27,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 문서편집, 넷플릭스시청,..."
1,2019.04.27,"[1, 동영상, 편집, 프리미어, 프로]"
2,2019.04.27,"[1, 사용, 용도, 업무용, 엑셀, 워드, 캐드]"
3,2019.04.26,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 인터넷, 유튜브, 한글,..."
4,2019.04.26,"[1, 사용, 용도, 사용하는, 프로그램, 게임명, 기재, 오피스, 간단한, 프로그..."


In [97]:
#data1.to_csv('date & purpose split.csv',encoding='ms949')